In [59]:
import re
import nltk
import spacy
import string
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

full_data = pd.read_csv("/content/insta_tags.csv", nrows=5000)
df = full_data[["text"]]
df["text"] = df["text"].astype(str)
full_data.head()

Unnamed: 0  hashed_userid  masked_username       location  following  \
0           0   2.878460e+17   *******itcombo  United States        398   
1           1   7.702490e+19  ********f_india          India          0   
2           2   6.342650e+19    *******a_Dass  Jaipur, India        653   
3           3   1.406950e+19     ******umar02          NOIDA        419   
4           4   1.581210e+19         ****sNow          India        371   

   followers  totaltweets usercreateddt       tweetid tweetcreatedts  ...  \
0         24        33582    16-08-2010  1.545280e+18        39:42.0  ...   
1        799        24993    21-09-2008  1.545280e+18        39:43.0  ...   
2        545          590    26-06-2021  1.545280e+18        39:45.0  ...   
3        921         1239    13-02-2010  1.545280e+18        39:47.0  ...   
4   10237893       818355    20-01-2011  1.545280e+18        39:47.0  ...   

                                                text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                            hashtags language favorite_count  \
0       [{'text': 'Shinzoabe', 'indices': [22, 32]}]       en              0   
1  [{'text': 'UPDATE', 'indices': [0, 7]}, {'text...       en              0   
2  [{'text': 'BreakingNews', 'indices': [0, 13]},...       en            413   
3  [{'text': 'ShinzoAbe', 'indices': [133, 143]},...       en              0   
4       [{'text': 'ShinzoAbe', 'indices': [31, 41]}]       en              8   

   is_retweet  original_tweet_id  in_reply_to_status_id  is_quote_status  \
0       False                0.0           0.000000e+00            False   
1       False                0.0           1.545280e+18            False   
2       False                0.0           0.000000e+00            False   
3       False                0.0           0.000000e+00            False   
4       False                0.0           0.000000e+00            False   

   quoted_status_id  extractedts  
0               0.0      49:21.9  
1               0.0      49:21.9  
2               0.0      49:21.9  
3               0.0      49:21.9  
4               0.0      49:21.8  

[5 rows x 21 columns]

Upper Case

In [60]:
df["upper_text"] = df["text"].str.upper()
df.head()

text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                          upper_text  
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...  
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...  
2  #BREAKINGNEWS SHOCKING \nA CLEAR PICTURE OF JA...  
3  SHOCKED TO HEAR THE NEWS OF THE ATTACK ON FORM...  
4  MOMENT WHEN FORMER JAPANESE PM #SHINZOABE WAS ...

Lower Case


In [61]:
df["lower_text"] = df["text"].str.lower()
df.head()

text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                          upper_text  \
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...   
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...   
2  #BREAKINGNEWS SHOCKING \nA CLEAR PICTURE OF JA...   
3  SHOCKED TO HEAR THE NEWS OF THE ATTACK ON FORM...   
4  MOMENT WHEN FORMER JAPANESE PM #SHINZOABE WAS ...   

                                          lower_text  
0  lol, friend made this #shinzoabe https://t.co/...  
1  #update from #japan since #shinzoabeshot \n\n#...  
2  #breakingnews shocking \na clear picture of ja...  
3  shocked to hear the news of the attack on form...  
4  moment when former japanese pm #shinzoabe was ...

Punctuation Removal

In [62]:
def remove_punctuation(text):
    return text.translate(str.maketrans('', '', Punctuations))

df["text_punct_removed"] = df["text"].apply(lambda text: remove_punctuation(text))
df.head()

text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                          upper_text  \
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...   
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...   
2  #BREAKINGNEWS SHOCKING \nA CLEAR PICTURE OF JA...   
3  SHOCKED TO HEAR THE NEWS OF THE ATTACK ON FORM...   
4  MOMENT WHEN FORMER JAPANESE PM #SHINZOABE WAS ...   

                                          lower_text  \
0  lol, friend made this #shinzoabe https://t.co/...   
1  #update from #japan since #shinzoabeshot \n\n#...   
2  #breakingnews shocking \na clear picture of ja...   
3  shocked to hear the news of the attack on form...   
4  moment when former japanese pm #shinzoabe was ...   

                                  text_punct_removed  
0  Lol friend made this Shinzoabe httpstcokjLTxkzWrz  
1  UPDATE from Japan since ShinzoAbeShot \n\nShin...  
2  BreakingNews Shocking \nA clear picture of Jap...  
3  Shocked to hear the news of the attack on form...  
4  Moment when former Japanese PM ShinzoAbe was s...

Freequent Words Removal

In [29]:
from collections import Counter
cnt = Counter()
for text in df["text"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('the', 3416),
 ('#ShinzoAbe', 2926),
 ('of', 2746),
 ('in', 2125),
 ('a', 2095),
 ('and', 1869),
 ('to', 1811),
 ('is', 1741),
 ('Shinzo', 1705),
 ('Abe', 1504)]

Tokenizer

In [44]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer_tok = ToktokTokenizer()
def tokenize_words(text):
    return tokenizer_tok.tokenize(text)

df["tokenized_tok"] = df["text"].apply(lambda text: tokenize_words(text))

print(df.head())

                                                text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                          upper_text  \
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...   
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...   
2  #BREAKINGNEWS SHOCKING \nA CLEAR PICTURE OF JA...   
3  SHOCKED TO HEAR THE NEWS OF THE ATTACK ON FORM...   
4  MOMENT WHEN FORMER JAPANESE PM #SHINZOABE WAS ...   

                                          lower_text  \
0  lol, friend made this #shinzoabe https://t.co/...   
1  #update from #japan since #shinzoabeshot \n\n#...   
2  #breakingnews shocking \na clear picture of ja...   
3  shocked to hear the news of the attack on form...   
4  moment when former japanese pm #shinzoabe w

lemmatization
```
# This is formatted as code
```



In [49]:
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
def lemmatize_text(text):
    tokens = word_tokenize(text)  
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return " ".join(lemmatized_tokens)
df['lemmatized_text'] = df['text'].apply(lemmatize_text)
df.head(2)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   

                                          upper_text  \
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...   
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...   

                                          lower_text  \
0  lol, friend made this #shinzoabe https://t.co/...   
1  #update from #japan since #shinzoabeshot \n\n#...   

                                  text_punct_removed  \
0  Lol friend made this Shinzoabe httpstcokjLTxkzWrz   
1  UPDATE from Japan since ShinzoAbeShot \n\nShin...   

                                       text_stemmed_  \
0  lol, friend made thi #shinzoab https://t.co/kj...   
1  #updat from #japan sinc #shinzoabeshot #shinzo...   

                                       tokenized_tok  \
0  [Lol, ,, friend, made, this, #Shinzoabe, https...   
1  [#UPDATE, from, #Japan, since, #ShinzoAbeShot,...   

                                     lemmatized_text  
0  Lol , friend made this # Shinzoabe http : //t....  
1  # UPDATE from # Japan since # ShinzoAbeShot # ...

Stemming

In [30]:
from nltk.stem.porter import PorterStemmer

Pstemmer = PorterStemmer()
def stem_words(text):
    return " ".join([Pstemmer.stem(word) for word in text.split()])

df["text_stemmed_"] = df["text"].apply(lambda text: stem_words(text))
df.head()

text  \
0  Lol, friend made this #Shinzoabe https://t.co/...   
1  #UPDATE from #Japan since #ShinzoAbeShot \n\n#...   
2  #BreakingNews Shocking \nA clear picture of Ja...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM #ShinzoAbe was ...   

                                          upper_text  \
0  LOL, FRIEND MADE THIS #SHINZOABE HTTPS://T.CO/...   
1  #UPDATE FROM #JAPAN SINCE #SHINZOABESHOT \n\n#...   
2  #BREAKINGNEWS SHOCKING \nA CLEAR PICTURE OF JA...   
3  SHOCKED TO HEAR THE NEWS OF THE ATTACK ON FORM...   
4  MOMENT WHEN FORMER JAPANESE PM #SHINZOABE WAS ...   

                                          lower_text  \
0  lol, friend made this #shinzoabe https://t.co/...   
1  #update from #japan since #shinzoabeshot \n\n#...   
2  #breakingnews shocking \na clear picture of ja...   
3  shocked to hear the news of the attack on form...   
4  moment when former japanese pm #shinzoabe was ...   

                                  text_punct_removed  \
0  Lol friend made this Shinzoabe httpstcokjLTxkzWrz   
1  UPDATE from Japan since ShinzoAbeShot \n\nShin...   
2  BreakingNews Shocking \nA clear picture of Jap...   
3  Shocked to hear the news of the attack on form...   
4  Moment when former Japanese PM ShinzoAbe was s...   

                                       text_stemmed_  
0  lol, friend made thi #shinzoab https://t.co/kj...  
1  #updat from #japan sinc #shinzoabeshot #shinzo...  
2  #breakingnew shock a clear pictur of japan' fo...  
3  shock to hear the news of the attack on former...  
4  moment when former japanes pm #shinzoab wa sho...

Spelling Correction

In [32]:
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.5 MB/s eta 0:00:00


In [33]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

text = "speling correctin"
correct_spellings(text)

'spelling correcting'

Removal Of URL

In [34]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [36]:
text = "proprocessing text refer : https://www.kaggle.com/datasets/seriousran/appletwittersentimenttexts"
remove_urls(text)

'proprocessing text refer : '